# Cleaning Data and Viewing it in QGIS

## Review some steps for reading and cleaning data from a csv file

In [1]:
# import python's csv module to open a csv file
import csv

In [2]:
# encapsulate the convert to integer functionality inside a new function
def extract_int_val(value):
    if not value == '':
        return int(float(value)) 
    else:
        return None

In [3]:
# create a new function to replace forward slashes and commas with hyphens
def clean_neighborhood(neighborhood_name):
    # you can daisy chain multiple string.replace() methods
    return neighborhood_name.replace('/', '-').replace(',', '')

In [4]:
# clean the data set by calling the cleaning functions and save the results to variables
rentals_cleaned = []
with open('Data/rents_raw.csv', 'r', encoding='utf-8') as csvfile:
    my_csv = csv.reader(csvfile)
    next(my_csv)
    for row in my_csv:
        neighborhood = clean_neighborhood(row[0])
        price = extract_int_val(row[1])
        bedrooms = extract_int_val(row[2])
        rentals_cleaned.append([neighborhood, price, bedrooms])      

# display our nested lists of data        
rentals_cleaned

[['foster city', 2495, 1],
 ['palo alto', 2695, None],
 ['brisbane', 3150, 2],
 ['palo alto', 2800, 2],
 ['san mateo', 2196, 1],
 ['santa clara', 3264, 3],
 ['san jose south', 2000, 2],
 ['sunnyvale', 4740, 3],
 ['inner sunset - UCSF', 3395, 2],
 ['richmond - seacliff', 2699, 1],
 ['SOMA - south beach', 3620, 1],
 ['dublin - pleasanton - livermore', 2025, 1],
 ['concord - pleasant hill - martinez', None, 2],
 ['hercules pinole san pablo el sob', 1795, 1],
 ['corte madera', 4299, 3]]

## What are the mean, min and max rent amounts?

In [5]:
count = 0 #initialize a counter
cumulative_rent = 0 # initialize cumulative_rent
max_val = 0 # set initial max at low value
min_val = 100000 # set initial min at high value

for listing in rentals_cleaned: # iterate over the rows 
    price_cleaned = listing[1] # create temp variable with a readable name since we did not store names
    if not (price_cleaned == None):
        count += 1
        cumulative_rent = cumulative_rent + price_cleaned
        if price_cleaned > max_val:
            max_val = price_cleaned
        
        if price_cleaned < min_val:
            min_val = price_cleaned

mean_rent = round(float(cumulative_rent) / count, 1)
            
print('Mean Rent is', mean_rent)
print('Min Rent is', min_val)
print('Max Rent is', max_val)

Mean Rent is 2940.9
Min Rent is 1795
Max Rent is 4740


## Now let's try looking at a big data file: one day of Craigslist rental listings for the entire U.S.

We will use a different function of the csv library for reading the file: DictReader

In [6]:
with open('Data/listings_one_day.csv', 'r', newline=None, encoding='utf-8') as csvfile: #newline=None enable universal new-lines
    my_csv = csv.DictReader(csvfile)
    print(my_csv.fieldnames)

['neighborhood', 'price', 'bedrooms', 'longitude', 'date', 'latitude', 'sqft']


In [7]:
rentals_one_day = []
with open('Data/listings_one_day.csv', 'r', newline=None, encoding='utf-8') as csvfile:
    my_csv = csv.DictReader(csvfile)
    missing_neighborhood = 0
    missing_price = 0
    missing_latitude = 0
    for row in my_csv:
        neighborhood = clean_neighborhood(row['neighborhood'])
        price = extract_int_val(row['price'])
        bedrooms = extract_int_val(row['bedrooms'])
        latitude = row['latitude']
        longitude = row['longitude']
        if neighborhood == '':
            missing_neighborhood += 1
        if price is None:
            missing_price += 1
        if latitude == '':
            missing_latitude += 1
        if (price is not None and (50 < price < 12000)) and neighborhood != '' and latitude != '':
            rentals_one_day.append([neighborhood, price, bedrooms, latitude, longitude])      

print('There are', len(rentals_one_day), 'total listings in this file')
print(missing_neighborhood, ' rows are missing neighborhood')
print(missing_price, 'rows are missing price')
print(missing_latitude, 'rows are missing latitude')

There are 126830 total listings in this file
14883  rows are missing neighborhood
25824 rows are missing price
118290 rows are missing latitude


In [8]:
count = 0 #initialize a counter
cumulative_rent = 0 # initialize cumulative_sum
max_val = 0 # set initial max at low value
min_val = 100000 # set initial min at high value

for listing in rentals_one_day: # iterate over the rows 
    price_cleaned = listing[1] # create temp variable with a readable name since we did not store names
    if not (price_cleaned == 0):
        count += 1
        cumulative_rent = cumulative_rent + price_cleaned
        if price_cleaned > max_val:
            max_val = price_cleaned
        
        if price_cleaned < min_val:
            min_val = price_cleaned

mean_rent = round(float(cumulative_rent) / count, 1)

print('Count is', count)
            
print('Mean Rent is', mean_rent)
print('Min Rent is', min_val)
print('Max Rent is', max_val)

Count is 126830
Mean Rent is 1392.6
Min Rent is 51
Max Rent is 11950


## Now write the data to a csv file that we can load into QGIS to visualize

In [9]:
with open('Data/listings_one_day_cleaned.csv', 'w', encoding='utf-8', newline='') as csv_file:
    clean_csv = csv.writer(csv_file)
    clean_csv.writerow(['neighborhood', 'price', 'bedrooms', 'latitude', 'longitude'])
    clean_csv.writerows(rentals_one_day)